# Konten

### Pendahuluan

## Tahap 1. Ikhtisar & Eksplorasi Data

## Tahap 2. Pra-pemrosesan Data
- 2.1 Gaya penulisan judul
- 2.2 Memeriksa Kualitas Data & Memperbaiki Data Anomali (Jika Ada)
- 2.3 Memeriksa Data Duplikat
- 2.4 Bekerja dengan Nilai-nilai yang hilang

## Tahap 3. Analisis Data Eksploratif (EDA)
- Pengkategorian Data
- Memeriksa Hipotesis

## Kesimpulan Umum

# Menganalisis risiko peminjam gagal membayar

Proyek Anda ialah menyiapkan laporan untuk bank bagian kredit. Anda harus mencari tahu pengaruh status perkawinan seorang nasabah dan jumlah anak terhadap probabilitas ketepatan waktu dalam melunasi pinjaman. Bank sudah memiliki beberapa data mengenai kelayakan kredit nasabah.

Laporan Anda akan dipertimbangkan pada saat membuat **penilaian kredit** untuk calon nasabah. **Penilaian kredit** digunakan untuk mengevaluasi kemampuan calon peminjam untuk melunasi pinjaman mereka.

Tujuan report ini untuk mendapatkan jawaban atas pertanyaan apakah ada hubungan antara ketepatan waktu dalam membayar pinnjaman dengan background nasabah, misalnya jumlah anak, status keluarga, tingkat pendapatan, dll.

Langkah-langkah yang akan diambil diantaranya:
1. Memeriksa nilai yang hilang & memperbaikinya dengan metode fillna
2. Memeriksa value unique di setiap kolom untuk memeriksa apakah ada keanehan & memperbaikinya dengan str.lower()
3. Memeriksa apakah ada value yang tidak masuk akal pada setiap kolom & memperbaikinya dengan nilai median/mean (tergantung kondisi value)
4. Membuat kelompok data untuk nilai kategoris dan numerik yang valuenya banyak untuk mempermudah analisis

# Ikhtisar & Eksplorasi Data

In [ ]:
# Memuat semua perpustakaan
import pandas as pd

In [ ]:
# Memuat semua perpustakaan
import numpy as np

In [ ]:
# muat data
data = pd.read_csv('/datasets/credit_scoring_eng.csv') 

**Deskripsi Data**
- *`children`* - jumlah anak dalam keluarga
- *`days_employed`* - pengalaman kerja dalam hari
- *`dob_years`* - usia klien dalam tahun
- *`education`* - pendidikan klien
- *`education_id`* - tanda pengenal pendidikan
- *`family_status`* - status perkawinan
- *`family_status_id`* - tanda pengenal status perkawinan
- *`gender`* - jenis kelamin klien
- *`income_type`* - jenis pekerjaan
- *`debt`* - apakah klien memiliki hutang pembayaran pinjaman
- *`total_income`* - pendapatan bulanan
- *`purpose`* - tujuan mendapatkan pinjaman

In [ ]:
# Mari kita lihat berapa banyak baris dan kolom yang dimiliki oleh dataset kita
data.shape

(21525, 12)

In [ ]:
# mari menampilkan N baris pertama

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Data sample diatas terlihat bermasalah, diantaranya:
1. Kolom `days_employed` bernilai negatif dimana seharusnya bekerja tidak mungkin negatif.
2. Kolom `days_employed` jumlah hari kerjanya ada yang tidak masuk akal 340266 hari / 365 hari = 932 tahun.
3. Kolom `education` memiliki unique value yang bedanya pada gaya penulisan huruf kapital, huruf kecil, dstnya.


In [ ]:
# Mendapatkan informasi data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Pada data ini, terdapat nilai-nilai yang hilang pada kolom days_employed dan total income dengan jumlah yang sama. Kita pastikan pada sel berikut.

In [ ]:
# Mari kita melihat tabel yang telah difilter dengan nilai yang hilang di kolom pertama dengan data yang hilang
data.loc[data['days_employed'].isna()]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Dan, ya, memang terlihat nilang yang hilang di kolom `days_employed` simetris dengan kolom `total_income`. Mungkin saja terjadi error system pada saat data diinput.

In [ ]:
# Mari kita menerapkan beberapa persyaratan untuk memfilter data dan melihat jumlah baris dalam tabel yang difilter.

data_null = data.loc[data['days_employed'].isna() & data['total_income'].isna()]
data_null

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Kesimpulan menengah**

Jumlah nilai yang hilang pada kolom `days_employed` & `total_income` saling beririsan dengan jumlah 2174 rows.

Selanjutnya, untuk lebih memastikan, kita akan mengecek persentase jumlah baris atas nilai yang hilang terhadap keseluruhan data.

In [ ]:
# Mari kita memeriksa klien yang tidak memiliki data tentang karakteristik yang teridentifikasi dan kolom dengan nilai yang hilang
data.isna().sum() / len(data) * 100


children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64

In [ ]:
# Memeriksa distribusi 
# Membandingkan value antara data_null dan data asli

categorical_columns = ['children', 'education', 'family_status', 'gender', 'purpose', 'income_type']

for i in categorical_columns:
    print('\nIni adalah unique value kolom %s dalam data_null:'%i)
    print (data_null[i].unique())
    print('\nIni adalah unique value kolom %s dalam data asli:'%i)
    print (data[i].unique())


Ini adalah unique value kolom children dalam data_null:
[ 0  1  2 -1  3  4 20  5]

Ini adalah unique value kolom children dalam data asli:
[ 1  0  3  2 -1  4 20  5]

Ini adalah unique value kolom education dalam data_null:
['secondary education' "bachelor's degree" 'SECONDARY EDUCATION'
 'some college' 'Secondary Education' 'Some College' "Bachelor's Degree"
 'SOME COLLEGE' 'primary education' "BACHELOR'S DEGREE"
 'Primary Education' 'PRIMARY EDUCATION']

Ini adalah unique value kolom education dalam data asli:
["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']

Ini adalah unique value kolom family_status dalam data_null:
['civil partnership' 'married' 'unmarried' 'widow / widower' 'divorced']

Ini adalah unique value kolom family_status dalam data 

Temuan pada kolom `children` adalah nilai minus dan 20, perlu dipertanyakan apakah terdapat error pada data. Sementara untuk kolom `education` dan `purpose` terdapat beberapa nilai yang sama namun ditulis dengan gaya penulisan yang beragam. Kita perlu betulkan nanti sehingga unique value nya bukan dari penulisan gaya yang beragam padahal value nya sama. 

**Kemungkinan penyebab hilangnya nilai dalam data**

Tidak ditemukan pola tertentu jika kita membandingkan data asli dengan data_null. Sepertinya memang datanya yang memiliki error.

In [ ]:
# Memeriksa distribusi di seluruh dataset
# Sudah dilakukan di sel sebelumnya

**Kesimpulan menengah**

Overall, data asli mirip dengan data_null yang sudah difilter hanya berisi data yang memiliki nilai yang hilang. Hanya ada perbedaan minor pada kolom `education` di data_null tidak memiliki value `graduate degree` dan pada kolom `gender` tidak memiliki value `XNA`.

In [ ]:
# Periksa penyebab dan pola lain yang dapat mengakibatkan nilai yang hilang
# Pada kolom education di data_null tidak memiliki value graduate degree
# Pada kolom gender tidak memiliki value XNA

categorical_columns = ['children', 'education', 'family_status', 'gender', 'purpose', 'income_type']

for i in categorical_columns:
    print('\nIni adalah unique value kolom %s dalam data_null:'%i)
    print (data_null[i].unique())
    print('\nIni adalah unique value kolom %s dalam data asli:'%i)
    print (data[i].unique())


Ini adalah unique value kolom children dalam data_null:
[ 0  1  2 -1  3  4 20  5]

Ini adalah unique value kolom children dalam data asli:
[ 1  0  3  2 -1  4 20  5]

Ini adalah unique value kolom education dalam data_null:
['secondary education' "bachelor's degree" 'SECONDARY EDUCATION'
 'some college' 'Secondary Education' 'Some College' "Bachelor's Degree"
 'SOME COLLEGE' 'primary education' "BACHELOR'S DEGREE"
 'Primary Education' 'PRIMARY EDUCATION']

Ini adalah unique value kolom education dalam data asli:
["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']

Ini adalah unique value kolom family_status dalam data_null:
['civil partnership' 'married' 'unmarried' 'widow / widower' 'divorced']

Ini adalah unique value kolom family_status dalam data 

**Kesimpulan menengah**

Tidak ditemukan pola tertentu, sepertinya mungkin saja hanya sistem yang error.

In [ ]:
# Memeriksa pola lainnya - jelaskan pola tersebut
# Sepertinya tidak ditemukan pola tertentu, mungkin saja hanya sistem yang error.

**Kesimpulan**

Tidak ditemukan adanya suatu pola. Sepertinya memang sistem error sehingga datanya menjadi seperti ini.
Untuk itu, kita akan melakukan pendekatan median atau mean (mana yang lebih sesuai) untuk mengatasi nilai-nilai yang hilang pada pada langkah selanjutnya.
Selain itu kita juga akan memeriksa kolom lain untuk menemukan apakah ada data bermasalah, seperti data duplikat, register yang berbeda-beda, value yang tidak masuk akal, dll.

# Pra-pemrosesan Data

## Gaya penulisan judul

In [ ]:
# Mari kita lihat semua nilai di kolom education untuk memeriksa ejaan apa yang perlu diperbaiki
sorted(data['education'].unique())

["BACHELOR'S DEGREE",
 "Bachelor's Degree",
 'GRADUATE DEGREE',
 'Graduate Degree',
 'PRIMARY EDUCATION',
 'Primary Education',
 'SECONDARY EDUCATION',
 'SOME COLLEGE',
 'Secondary Education',
 'Some College',
 "bachelor's degree",
 'graduate degree',
 'primary education',
 'secondary education',
 'some college']

In [ ]:
# Perbaiki register jika diperlukan
data['education'] = data['education'].str.lower()

In [ ]:
# Memeriksa semua nilai di kolom untuk memastikan kita telah memperbaikinya
# Setelah diperbaiki, nilai di kolom education terlihat lebih jelas dimana hanya ada 4 jenis pendidikan

sorted(data['education'].unique())

["bachelor's degree",
 'graduate degree',
 'primary education',
 'secondary education',
 'some college']

## Memeriksa Kualitas Data & Memperbaiki Data Anomali (Jika Ada)

### Kolom `children`

In [ ]:
# Mari kita lihat distribusi nilai pada kolom `children`
print(data['children'].value_counts())


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [ ]:
# Menghitung jumlah value 20 dan -1 di kolom children

print(len(data.loc[(data['children'] == 20) | (data['children'] == -1)]))

123


In [ ]:
# Menghitung persentase jumlah value 20 dan -1 di kolom children terhadap keseluruhan data

len(data.loc[(data['children'] == 20) | (data['children'] == -1)]) / len(data) *100

0.5714285714285714

Ada masalah pada kolom children, yaitu value 20 dan -1, yang berjumlah 0.5% dari total data. Bisa jadi ini terjadi karena ada error sistem sehingga data yang disediakan menjadi aneh. Karena total bermasalah tidak siginifikan, maka akan diperbaiki dengan menggantinya menjadi 0.

In [ ]:
# [perbaiki data berdasarkan keputusan Anda]
data.loc[data['children'] == 20, 'children'] = 0
data.loc[data['children'] == -1, 'children'] = 0

In [ ]:
# Periksa kembali kolom `children` untuk memastikan semua telah diperbaiki
data['children'].value_counts()


0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

### Kolom `days_employed`

In [ ]:
# Temukan data yang bermasalah di `days_employed`, jika terdapat masalah, dan hitung persentasenya

print(data['days_employed'] / 365)


0        -23.116912
1        -11.026860
2        -15.406637
3        -11.300677
4        932.235814
            ...    
21520    -12.409087
21521    942.294258
21522     -5.789991
21523     -8.527347
21524     -5.437007
Name: days_employed, Length: 21525, dtype: float64


In [ ]:
# hitung persentase yang bermasalah

len(data.loc[(data['days_employed'] < 0)]) / len(data) * 100

73.89547038327527

In [ ]:
# Atasi nilai yang bermasalah, jika ada
# 1. mengatasi nilai yang minus karena lama bekerja tidak mungkin minus
# 2. mengatasi nilai yang terlalu besar / tidak masuk akal > 16790 hari = 46 tahun
    # asumsi mulai  bekerja umur 19 tahun dan pensiun max umur 65 tahun
    # untuk dob_years yang >65 tahun, nilai days_employed nya diganti menjadi 16790 (asumsi max pensiun umur 65 tahun)
    # untuk dob_years yang 0 tahun, maka nilai days_employed diganti dengan median days_employed
    
data['days_employed'] = data['days_employed'].abs()
data.loc[(data['days_employed'] > 16790) & (data['dob_years'] >= 19) & (data['dob_years'] < 66), 'days_employed'] = (data['dob_years'] - 19) * 365
data.loc[(data['dob_years'] > 65), 'days_employed'] = 16790
data.loc[(data['dob_years'] == 0), 'days_employed'] = data['days_employed'].median()


In [ ]:
# Periksa hasilnya - pastikan telah diperbaiki

print(data['days_employed'] / 365)

0        23.116912
1        11.026860
2        15.406637
3        11.300677
4        34.000000
           ...    
21520    12.409087
21521    46.000000
21522     5.789991
21523     8.527347
21524     5.437007
Name: days_employed, Length: 21525, dtype: float64


In [ ]:
# Untuk memeriksa hasil tidak ada lagi days_employed dengan nilai negatif
print(len(data.loc[(data['days_employed'] < 0)]) / len(data) * 100)

0.0


In [ ]:
# Untuk memeriksa hasil tidak ada lagi days_employed dengan nilai terlalu besar / tidak masuk akal
len(data.loc[(data['days_employed'] > 16790)]) / len(data) * 100

0.0

### Kolom `dob_years`

In [ ]:
# Periksa `dob_years` untuk nilai yang mencurigakan dan hitung persentasenya

print(data['dob_years'].value_counts().sort_values())

75      1
74      6
73      8
19     14
72     33
20     51
71     58
70     65
69     85
68     99
0     101
21    111
67    167
22    183
66    183
65    194
23    254
24    264
64    265
63    269
62    352
61    355
25    357
60    377
26    408
55    443
59    444
51    448
53    459
57    460
58    461
46    475
54    479
47    480
52    484
56    487
27    493
45    497
28    503
49    508
32    510
43    513
50    514
37    537
48    538
30    540
29    545
44    547
36    555
31    560
39    573
33    581
42    597
38    598
34    603
41    607
40    609
35    617
Name: dob_years, dtype: int64


In [ ]:
# menhitung persentase nilai mencurigakan
len(data.loc[(data['dob_years'] < 19)]) / len(data) * 100

0.4692218350754936

Persentase nilai bermasalah pada umur < 19 tahun hanya 0.46%. Namun, kita tetap akan menggantinya menjadi minimum 19 tahun (minimum umur yang ada dalam dataset adalah 19 tahun). Karena umur ini akan digunakan juga untuk mengatasi masalah diatas pada kolom days_employed yang memiliki jumlah hari bekerja tidak masuk akal > 16790 hari.

In [ ]:
# Atasi masalah pada kolom `dob_years`

data.loc[data['dob_years'] == 0, 'dob_years'] = 19

In [ ]:
# Periksa hasilnya - pastikan telah diperbaiki
data['dob_years'].value_counts().sort_values()

75      1
74      6
73      8
72     33
20     51
71     58
70     65
69     85
68     99
21    111
19    115
67    167
66    183
22    183
65    194
23    254
24    264
64    265
63    269
62    352
61    355
25    357
60    377
26    408
55    443
59    444
51    448
53    459
57    460
58    461
46    475
54    479
47    480
52    484
56    487
27    493
45    497
28    503
49    508
32    510
43    513
50    514
37    537
48    538
30    540
29    545
44    547
36    555
31    560
39    573
33    581
42    597
38    598
34    603
41    607
40    609
35    617
Name: dob_years, dtype: int64

### Kolom `family_status`

In [ ]:
# Mari kita lihat nilai untuk kolom

data['family_status'].value_counts()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

Tidak ada data bermasalah pada kolom `family_status`.

### Kolom `gender`

In [ ]:
# Mari kita melihat nilainya di kolom
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [ ]:
# Atasi nilai-nilai yang bermasalah, jika ada
data.loc[data['gender'] == 'XNA', 'gender'] = 'unknown'

In [ ]:
# Periksa hasilnya - pastikan telah diperbaiki
data['gender'].value_counts()


F          14236
M           7288
unknown        1
Name: gender, dtype: int64

### Kolom `income_type`

In [ ]:
# Mari kita lihat nilai dalam kolom
data['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

Tidak ada data bermasalah pada kolom `income_type`.

## Memeriksa Data Duplikat

In [ ]:
# Memeriksa duplikat
data.duplicated().sum()


71

In [ ]:
# Atasi duplikat, jika ada
data = data.drop_duplicates().reset_index(drop=True)


In [ ]:
# Terakhir periksa apakah kita masih memiliki duplikat
data.duplicated().sum()

0

In [ ]:
# Periksa ukuran dataset yang sekarang Anda miliki setelah manipulasi pertama yang telah lakukan
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19419 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21454 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
# Persentase perubahan kolom days_employed setelah manipulasi pertama diatas 
(19419 - 19351) / 19351

0.0035140302826727303

**Ksimpulan Menengah**

Sekarang dataset kita memiliki perubahan, dimana jumlah kolom `days_employed` tidak lagi sama dengan `total_income`. Hal ini dikarenakan kita sudah memanipulasi data `days_employed` yang jika umurnya 0, maka `days_employed` diganti dengan nilai median `days_employed`. % perubahannya sangat kecil, hanya 0.35%.

## Bekerja dengan nilai yang hilang

In [ ]:
# Temukan dictionary

sorted(data['dob_years'].unique())

[19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75]

### Memperbaiki nilai yang hilang di `total_income`

Kolom `total_income` memiliki nilai yang hilang dan akan diatasi dengan menggantinya dengan nilai median ke kolom baru.


In [ ]:
# Mari menulis fungsi untuk menghitung kategori usia
data['dob_years'] = pd.to_numeric(data['dob_years'], errors='coerce')

age = data['dob_years']
def assign_age_group(age):
    if age < 26:
        return '1-25 tahun'
    elif age < 36:
        return '26-35 tahun'
    elif age < 46:
        return '36-45 tahun'
    elif age < 56:
        return '46-55 tahun'
    elif age < 66:
        return '56-65 tahun'
    else:
        return '66 tahun keatas'

In [ ]:
# Lakukan pengujian apakah fungsi bekerja atau tidak
assign_age_group(25)

'1-25 tahun'

In [ ]:
# Membuat kolom baru berdasarkan fungsi
    
data['age_group'] = data['dob_years'].apply(assign_age_group) 
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,36-45 tahun
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,36-45 tahun
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,26-35 tahun
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,26-35 tahun
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-55 tahun


In [ ]:
# Memeriksa bagaimana nilai di dalam kolom baru

data['age_group'].value_counts()

36-45 tahun        5620
26-35 tahun        5351
46-55 tahun        4814
56-65 tahun        3634
1-25 tahun         1333
66 tahun keatas     702
Name: age_group, dtype: int64

In [ ]:
# Membuat tabel tanpa nilai yang hilang dan menampilkan beberapa barisnya untuk memastikan semuanya berjalan dengan baik
data_withoutnull = data.loc[~(data['days_employed'].isna()) & ~(data['total_income'].isna())]
data_withoutnull

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,36-45 tahun
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,36-45 tahun
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,26-35 tahun
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,26-35 tahun
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-55 tahun
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,36-45 tahun
21450,0,16790.000000,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,66 tahun keatas
21451,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,36-45 tahun
21452,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,36-45 tahun


In [ ]:
# Perhatikan nilai rata-rata untuk pendapatan berdasarkan faktor yang telah Anda identifikasi
data_withoutnull.groupby('income_type')['total_income'].mean()


income_type
business                       32386.793835
civil servant                  27343.729582
employee                       25820.841683
entrepreneur                   79866.103000
paternity / maternity leave     8612.661000
retiree                        21940.394503
student                        15712.260000
unemployed                     21014.360500
Name: total_income, dtype: float64

In [ ]:
# Perhatikan nilai median untuk pendapatan berdasarkan faktor yang telah Anda identifikasi
data_withoutnull.groupby('income_type')['total_income'].median()


income_type
business                       27577.2720
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64

Kita akan memakai median untuk mengganti nilai yang hilang karena income setiap orang bervariasi, jika menggunakan mean dirasa kurang tepat untuk merepresentasikan nilai yang hilang.

In [ ]:
#  Tulis fungsi yang akan kita gunakan untuk mengisi nilai yang hilang

def data_imputation(data, column_filter, column_selected, column_target, rules):
    data[column_target] = data[column_selected]

    # Looping di setiap value group category
    for rule in rules:
        # get median
        median = data.loc[(data[column_filter]==rule) & ~(data[column_selected].isna()), column_target].median()

        # change missing value
        data.loc[(data[column_filter]==rule) & (data[column_selected].isna()), column_target] = median

    return data

In [ ]:
# Memeriksa bagaimana nilai di dalam kolom baru
rules = data['income_type'].unique()

data_imputation(data, column_filter='income_type', column_selected='total_income', column_target='total_income_clean', rules=rules)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_clean
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,36-45 tahun,40620.102
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,36-45 tahun,17932.802
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,26-35 tahun,23341.752
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,26-35 tahun,42820.568
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-55 tahun,25378.572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,36-45 tahun,35966.698
21450,0,16790.000000,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,66 tahun keatas,24959.969
21451,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,36-45 tahun,14347.610
21452,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,36-45 tahun,39054.888


In [ ]:
# Periksa bahwa fungsi bekerja
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_clean
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,36-45 tahun,40620.102
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,36-45 tahun,17932.802
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,26-35 tahun,23341.752
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,26-35 tahun,42820.568
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-55 tahun,25378.572
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,26-35 tahun,40922.170
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,36-45 tahun,38484.156
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,46-55 tahun,21731.829
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,26-35 tahun,15337.093
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,36-45 tahun,23108.150


In [ ]:
# Periksa apakah kita mendapatkan kesalahan
data['total_income_clean'].isna().sum()

0

In [ ]:
# Memeriksa jumlah entri di kolom
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   children            21454 non-null  int64  
 1   days_employed       19419 non-null  float64
 2   dob_years           21454 non-null  int64  
 3   education           21454 non-null  object 
 4   education_id        21454 non-null  int64  
 5   family_status       21454 non-null  object 
 6   family_status_id    21454 non-null  int64  
 7   gender              21454 non-null  object 
 8   income_type         21454 non-null  object 
 9   debt                21454 non-null  int64  
 10  total_income        19351 non-null  float64
 11  purpose             21454 non-null  object 
 12  age_group           21454 non-null  object 
 13  total_income_clean  21454 non-null  float64
dtypes: float64(3), int64(5), object(6)
memory usage: 2.3+ MB


###  Memperbaiki nilai di `days_employed`

In [ ]:
# Distribusi median dari `days_employed` berdasarkan parameter yang Anda identifikasi

data.groupby('age_group')['days_employed'].median()

age_group
1-25 tahun           885.674587
26-35 tahun         1350.995300
36-45 tahun         1917.145342
46-55 tahun         2886.495612
56-65 tahun        14235.000000
66 tahun keatas    16790.000000
Name: days_employed, dtype: float64

In [ ]:
# Distribusi rata-rata dari `days_employed` berdasarkan parameter yang Anda identifikasi

data.groupby('age_group')['days_employed'].mean()

age_group
1-25 tahun          1032.172063
26-35 tahun         1689.775100
36-45 tahun         2559.939899
46-55 tahun         4596.744180
56-65 tahun        10818.555121
66 tahun keatas    16790.000000
Name: days_employed, dtype: float64

Kita memilih menggunakan median untuk mengganti nilai yang hilang pad kolom `days_employed` karena value yang bervariasi pada kolom `days_employed` tidak memungkinkan memakai mean.

In [ ]:
# Mari tulis fungsi yang menghitung rata-rata atau median (tergantung keputusan Anda) berdasarkan parameter yang Anda identifikasi

def data_imputation(data, column_filter, column_selected, rules):
    # Looping di setiap value group category
    for rule in rules:
        # get median
        median = data.loc[(data[column_filter]==rule) & ~(data[column_selected].isna()), column_selected].median()

        # change missing value
        data.loc[(data[column_filter]==rule) & (data[column_selected].isna()), column_selected] = median

    return data

In [ ]:
# Periksa bahwa fungsi bekerja
rules = data['age_group'].unique()
data_imputation(data, column_filter='age_group', column_selected='days_employed', rules=rules)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_clean
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,36-45 tahun,40620.102
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,36-45 tahun,17932.802
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,26-35 tahun,23341.752
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,26-35 tahun,42820.568
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-55 tahun,25378.572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,36-45 tahun,35966.698
21450,0,16790.000000,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,66 tahun keatas,24959.969
21451,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,36-45 tahun,14347.610
21452,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,36-45 tahun,39054.888


In [ ]:
# Terapkan fungsi ke income_type
rules = data['income_type'].unique()

data_imputation(data, column_filter='income_type', column_selected='total_income', rules=rules)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_clean
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,36-45 tahun,40620.102
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,36-45 tahun,17932.802
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,26-35 tahun,23341.752
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,26-35 tahun,42820.568
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-55 tahun,25378.572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,36-45 tahun,35966.698
21450,0,16790.000000,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,66 tahun keatas,24959.969
21451,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,36-45 tahun,14347.610
21452,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,36-45 tahun,39054.888


In [ ]:
# Periksa bahwa fungsi bekerja
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_clean
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,36-45 tahun,40620.102
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,36-45 tahun,17932.802
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,26-35 tahun,23341.752
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,26-35 tahun,42820.568
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-55 tahun,25378.572
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,26-35 tahun,40922.170
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,36-45 tahun,38484.156
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,46-55 tahun,21731.829
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,26-35 tahun,15337.093
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,36-45 tahun,23108.150


In [ ]:
# Periksa entri di semua kolom - pastikan kita memperbaiki semua nilai yang hilang
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   children            21454 non-null  int64  
 1   days_employed       21454 non-null  float64
 2   dob_years           21454 non-null  int64  
 3   education           21454 non-null  object 
 4   education_id        21454 non-null  int64  
 5   family_status       21454 non-null  object 
 6   family_status_id    21454 non-null  int64  
 7   gender              21454 non-null  object 
 8   income_type         21454 non-null  object 
 9   debt                21454 non-null  int64  
 10  total_income        21454 non-null  float64
 11  purpose             21454 non-null  object 
 12  age_group           21454 non-null  object 
 13  total_income_clean  21454 non-null  float64
dtypes: float64(3), int64(5), object(6)
memory usage: 2.3+ MB


# Analisis Data Eksploratif (EDA)

Pada bagian Hipotesis kita akan menjawab pertanyaan-pertanyaan berikut ini:
1. Apakah terdapat hubungan antara memiliki anak dan probabilitas seseorang melakukan gagal bayar pinjaman?
2. Apakah terdapat hubungan antara status perkawinan dan  probabilitas seseorang melakukan gagal bayar pinjaman?
3. Apakah terdapat hubungan antara tingkat pendapatan dan probabilitas seseorang melakukan gagal bayar pinjaman?
4. Bagaimana perbedaan tujuan pinjaman memengaruhi probabilitas seseorang melakukan gagal bayar pinjaman?

### Pengkategorian Data

In [ ]:
# Tampilkan nilai data yang Anda pilih untuk pengkategorian

data[['children', 'family_status', 'purpose']]


,children,family_status,purpose
0,1,married,purchase of the house
1,1,married,car purchase
2,0,married,purchase of the house
3,3,married,supplementary education
4,0,civil partnership,to have a wedding
...,...,...,...
21449,1,civil partnership,housing transactions
21450,0,married,purchase of a car
21451,1,civil partnership,property
21452,3,married,buying my own car


In [ ]:
# Periksa nilai unik
# pada kolom age_group

sorted(data['children'].unique())

[0, 1, 2, 3, 4, 5]

In [ ]:
# Periksa nilai unik
# pada kolom children

sorted(data['family_status'].unique())

['civil partnership', 'divorced', 'married', 'unmarried', 'widow / widower']

In [ ]:
# Periksa nilai unik
# pada kolom purpose

sorted(data['purpose'].unique())

['building a property',
 'building a real estate',
 'buy commercial real estate',
 'buy real estate',
 'buy residential real estate',
 'buying a second-hand car',
 'buying my own car',
 'buying property for renting out',
 'car',
 'car purchase',
 'cars',
 'construction of own property',
 'education',
 'getting an education',
 'getting higher education',
 'going to university',
 'having a wedding',
 'housing',
 'housing renovation',
 'housing transactions',
 'profile education',
 'property',
 'purchase of a car',
 'purchase of my own house',
 'purchase of the house',
 'purchase of the house for my family',
 'real estate transactions',
 'second-hand car purchase',
 'supplementary education',
 'to become educated',
 'to buy a car',
 'to get a supplementary education',
 'to have a wedding',
 'to own a car',
 'transactions with commercial real estate',
 'transactions with my real estate',
 'university education',
 'wedding ceremony']

Kelompok utama yang akan diidentifikasi adalah `debt` berdasarkan nilai unik `purpose` dan `total_income`.


In [ ]:
# Mari kita menulis fungsi untuk mengkategorikan data berdasarkan topik umum
# kategori untuk kolom debt

def replace_debt(value):
    if value == 1:
        return "history gagal bayar"
    else:
        return "history hutang lancar"

In [ ]:
# Buat kolom dengan kategori dan hitung nilainya
# Kolom baru untuk mengkategorikan kolom debt

data['debt_meaning'] = data['debt'].apply(replace_debt)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_clean,debt_meaning
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,36-45 tahun,40620.102,history hutang lancar
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,36-45 tahun,17932.802,history hutang lancar
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,26-35 tahun,23341.752,history hutang lancar
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,26-35 tahun,42820.568,history hutang lancar
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-55 tahun,25378.572,history hutang lancar


In [ ]:
# Mari kita menulis fungsi untuk mengkategorikan data berdasarkan topik umum
# kategori untuk kolom purpose karena nilai unik pada kolom purpose >10 dan beberapa value mengandung arti yang sama
# kelompok kategori dibuat ke kolom baru

data.loc[data['purpose'].isin(['building a property', 'building a real estate', 'buy commercial real estate', 'buy real estate', 'buy residential real estate', 'housing', 'housing transactions', 'purchase of my own house', 'purchase of the house', 'purchase of the house for my family', 'real estate transactions', 'transactions with commercial real estate', 'transactions with my real estate']), 'purpose_group'] = 'buy house'
data.loc[data['purpose'].isin(['having a wedding', 'to have a wedding', 'wedding ceremony']), 'purpose_group'] = 'wedding'
data.loc[data['purpose'].isin(['education', 'getting an education', 'getting higher education', 'going to university', 'profile education', 'supplementary education', 'to become educated', 'to get a supplementary education', 'university education']), 'purpose_group'] = 'education'
data.loc[data['purpose'].isin(['construction of own property', 'housing renovation']), 'purpose_group'] = 'housing renovation'
data.loc[data['purpose'].isin(['buying property for renting out', 'property']), 'purpose_group'] = 'buy property'
data.loc[data['purpose'].isin(['buying a second-hand car', 'buying my own car', 'car', 'car purchase', 'cars', 'purchase of a car', 'second-hand car purchase', 'to buy a car', 'to own a car']), 'purpose_group'] = 'car purchase'


In [ ]:
# Memeriksa fungsi bekerja
data.loc[data['purpose'] == 'construction of own property'].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_clean,debt_meaning,purpose_group
15,1,972.364419,26,secondary education,1,married,0,F,employee,0,18691.345,construction of own property,26-35 tahun,18691.345,history hutang lancar,housing renovation
27,0,529.191635,28,bachelor's degree,0,married,0,M,employee,0,49415.837,construction of own property,26-35 tahun,49415.837,history hutang lancar,housing renovation
28,1,717.274324,26,bachelor's degree,0,married,0,F,employee,0,30058.118,construction of own property,26-35 tahun,30058.118,history hutang lancar,housing renovation
48,0,3341.067886,45,secondary education,1,married,0,F,employee,0,25930.483,construction of own property,36-45 tahun,25930.483,history hutang lancar,housing renovation
105,0,2098.626296,62,secondary education,1,widow / widower,2,F,employee,0,12301.470,construction of own property,56-65 tahun,12301.470,history hutang lancar,housing renovation


In [ ]:
# Melihat distribusi
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   children            21454 non-null  int64  
 1   days_employed       21454 non-null  float64
 2   dob_years           21454 non-null  int64  
 3   education           21454 non-null  object 
 4   education_id        21454 non-null  int64  
 5   family_status       21454 non-null  object 
 6   family_status_id    21454 non-null  int64  
 7   gender              21454 non-null  object 
 8   income_type         21454 non-null  object 
 9   debt                21454 non-null  int64  
 10  total_income        21454 non-null  float64
 11  purpose             21454 non-null  object 
 12  age_group           21454 non-null  object 
 13  total_income_clean  21454 non-null  float64
 14  debt_meaning        21454 non-null  object 
 15  purpose_group       21454 non-null  object 
dtypes: f

In [ ]:
# Periksa nilai unik pada kolom purpose_group

sorted(data['purpose_group'].unique())

['buy house',
 'buy property',
 'car purchase',
 'education',
 'housing renovation',
 'wedding']

Pengkategorian kelompok numerik dari kolom `total_income` dengan mengelompokkan pendapatan ke beberapa rentang pendapatan, karena pendapatan nilainya bervariasi. Sehingga bisa membantu kita dalam mengambil keputusan dalam memberikan pinjaman.

In [ ]:
# Membuat fungsi untuk pengkategorian menjadi kelompok numerik yang berbeda berdasarkan rentang

income = data['total_income']
def assign_income_group(income):
    if income < 10000:
        return 'below 10k'
    elif income >= 10000 and income < 25000:
        return '10k-25k'
    elif income >= 25000 and income < 50000:
        return '25k-50k'
    elif income >= 50000 and income < 100000:
        return '50k-100k'
    elif income >= 100000 and income < 300000:
        return '100k-300k'
    else:
        return 'above 300k'

In [ ]:
# Membuat kolom dengan kategori
data['income_group'] = data['total_income'].apply(assign_income_group) 
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_clean,debt_meaning,purpose_group,income_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,36-45 tahun,40620.102,history hutang lancar,buy house,25k-50k
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,36-45 tahun,17932.802,history hutang lancar,car purchase,10k-25k
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,26-35 tahun,23341.752,history hutang lancar,buy house,10k-25k
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,26-35 tahun,42820.568,history hutang lancar,education,25k-50k
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-55 tahun,25378.572,history hutang lancar,wedding,25k-50k


In [ ]:
# Menghitung setiap nilai kategori untuk melihat pendistribusian

data['debt_meaning'].value_counts()

history hutang lancar    19713
history gagal bayar       1741
Name: debt_meaning, dtype: int64

In [ ]:
# Menghitung setiap nilai kategori untuk melihat pendistribusian
data['purpose_group'].value_counts()

buy house             8285
car purchase          4306
education             4013
wedding               2324
buy property          1284
housing renovation    1242
Name: purpose_group, dtype: int64

In [ ]:
# Menghitung setiap nilai kategori untuk melihat pendistribusian
data['income_group'].value_counts()

10k-25k       11423
25k-50k        7784
50k-100k       1222
below 10k       926
100k-300k        97
above 300k        2
Name: income_group, dtype: int64

## Memeriksa Hipotesis

1. Apakah terdapat hubungan antara memiliki anak dan probabilitas seseorang melakukan gagal bayar pinjaman?
2. Apakah terdapat hubungan antara status perkawinan dan  probabilitas seseorang melakukan gagal bayar pinjaman?
3. Apakah terdapat hubungan antara tingkat pendapatan dan probabilitas seseorang melakukan gagal bayar pinjaman?
4. Bagaimana perbedaan tujuan pinjaman memengaruhi probabilitas seseorang melakukan gagal bayar pinjaman?

### Apakah terdapat korelasi antara memiliki anak dengan membayar kembali tepat waktu?

In [ ]:
# Periksa data anak dan membayar kembali dengan tepat waktu

data[['children', 'debt']]

,children,debt
0,1,0
1,1,0
2,0,0
3,3,0
4,0,0
...,...,...
21449,1,0
21450,0,0
21451,1,1
21452,3,1


In [ ]:
# Menghitung tarif otomatis berdasarkan jumlah anak
# default_rate = persentase gagal bayar

data_pivot_by_children = data.pivot_table(
    values='debt_meaning',
    index='children',
    columns='debt',
    aggfunc='count',
    margins=True,
)

data_pivot_by_children['Proporsi'] = round(data_pivot_by_children[1] / data_pivot_by_children['All'] * 100, 2)
data_pivot_by_children

debt,0,1,All,Proporsi
children,,,,
0,13142.0,1072.0,14214,7.54
1,4364.0,444.0,4808,9.23
2,1858.0,194.0,2052,9.45
3,303.0,27.0,330,8.18
4,37.0,4.0,41,9.76
5,9.0,NaN,9,NaN
All,19713.0,1741.0,21454,8.12


**Kesimpulan**

Melihat data diatas, jumlah anak berpengaruh kepada kegagalan debitur dalam membayar hutang mulai dari 8.18%. Sementara, nasabah tanpa anak memiliki jumlah terbanyak yang gagal membayar hutang meski persentasenya lebih kecil sedikit, yaitu 7.54%. Jadi, menurut kami memiliki anak tidak signifikan berhubungan dengan tingkat kegagalan nasabah dalam membayar hutang. 

### Apakah terdapat korelasi antara status keluarga dengan membayar kembali tepat waktu?

In [ ]:
# Periksa data status keluarga dan membayar kembali dengan tepat waktu

data[['family_status', 'debt']]

,family_status,debt
0,married,0
1,married,0
2,married,0
3,married,0
4,civil partnership,0
...,...,...
21449,civil partnership,0
21450,married,0
21451,civil partnership,1
21452,married,1


In [ ]:
# Menghitung tarif otomatis berdasarkan status keluarga

data_pivot_by_family = data.pivot_table(
    values='debt_meaning',
    index='family_status',
    columns='debt',
    aggfunc='count',
    margins=True,
)

data_pivot_by_family['Proporsi'] = round(data_pivot_by_family[1] / data_pivot_by_family['All'] * 100, 2)
data_pivot_by_family

debt,0,1,All,Proporsi
family_status,,,,
civil partnership,3763,388,4151,9.35
divorced,1110,85,1195,7.11
married,11408,931,12339,7.55
unmarried,2536,274,2810,9.75
widow / widower,896,63,959,6.57
All,19713,1741,21454,8.12


**Kesimpulan**

Secara persentase, nasabah status `civil_partnership` memiliki tingkat gagal bayar paling tinggi. Sementara, nasabah dengan status married justru lebih tepat waktu dalam membayar hutang. Hal ini mungkin berhubungan dengan sudah adanya pengalaman dalam mengatur keuangan bagi mereka yang sudah menikah. Selain itu, nasabah dengan status widow/widower & divorced juga memiliki tingkat gagal bayar hutang yang rendah. 

### Apakah terdapat korelasi antara tingkat pendapatan dengan membayar kembali tepat waktu?

In [ ]:
# Periksa data tingkat pendapatan dan membayar kembali dengan tepat waktu

data[['income_group', 'debt']]

,income_group,debt
0,25k-50k,0
1,10k-25k,0
2,10k-25k,0
3,25k-50k,0
4,25k-50k,0
...,...,...
21449,25k-50k,0
21450,10k-25k,0
21451,10k-25k,1
21452,25k-50k,1


In [ ]:
# Menghitung tarif otomatis berdasarkan tingkat pendapatan

data_pivot_by_income = data.pivot_table(
    values='debt_meaning',
    index='income_group',
    columns='debt',
    aggfunc='count',
    margins=True,
)

data_pivot_by_income['Proporsi'] = round(data_pivot_by_income[1] / data_pivot_by_income['All'] * 100, 2)
data_pivot_by_income

debt,0,1,All,Proporsi
income_group,,,,
100k-300k,92,5,97,5.15
10k-25k,10445,978,11423,8.56
25k-50k,7171,613,7784,7.88
50k-100k,1136,86,1222,7.04
above 300k,1,1,2,50.00
below 10k,868,58,926,6.26
All,19713,1741,21454,8.12


**Kesimpulan**

Wow! Ternyata, penghasilan tinggi tidak menjamin seseorang membayar hutang tepat waktu, orang kaya mungkin memiliki lebih banyak hutang lagi, dibandingkan dengan orang berpenghasilan rendah. Atau bisa jadi dikarenakan orang kaya memiliki biaya gaya hidup yang tinggi sehingga berpengaruh kepada kegagalan membayar hutang mereka. Meskipun pada data ini, hanya berjumlah 2 orang, namun bayangkan apabila 50% nasabah Bank berasal dari kategori pendapatan >300k, berapa estimasi kerugian yang harus dicadangkan Bank apabila setengahnya benar-benar gagal bayar hutang :')

### Bagaimana tujuan kredit memengaruhi tarif otomatis?

In [ ]:
# Periksa persentase tarif otomatis untuk setiap tujuan kredit dan lakukan penganalisisan

data[['purpose_group', 'debt']]

,purpose_group,debt
0,buy house,0
1,car purchase,0
2,buy house,0
3,education,0
4,wedding,0
...,...,...
21449,buy house,0
21450,car purchase,0
21451,buy property,1
21452,car purchase,1


In [ ]:
# Periksa persentase tarif otomatis untuk setiap tujuan kredit
data_pivot_by_purpose = data.pivot_table(
    values='debt_meaning',
    index='purpose_group',
    columns='debt',
    aggfunc='count',
    margins=True,
)

data_pivot_by_purpose['Proporsi'] = round(data_pivot_by_purpose[1] / data_pivot_by_purpose['All'] * 100, 2)
data_pivot_by_purpose

debt,0,1,All,Proporsi
purpose_group,,,,
buy house,7674,611,8285,7.37
buy property,1190,94,1284,7.32
car purchase,3903,403,4306,9.36
education,3643,370,4013,9.22
housing renovation,1165,77,1242,6.20
wedding,2138,186,2324,8.00
All,19713,1741,21454,8.12


**Kesimpulan**

Nasabah dengan tujuan pinjaman pembelian mobil & pendidikan memiliki tingkat probabilitas gagal bayar paling tinggi, sementara orang-orang dengan tujuan merenovasi rumah memiliki tingkat probabilitas gagal bayar paling rendah. Mungkin, ini berhubungan dengan umur nasabah dan pengalaman kerja mereka, sehingga income mereka masih terbatas untuk memenuhi kewajiban mereka.

# Kesimpulan Umum 

1. Nilai yang hilang pada kolom `days_employed` & `total_income` simetris yaitu 2174 rows (10% dari data). Mungkin saja ini adalah kesalahan sistem. Tidak ditemukan pola dalam hilangnya data, karena setelah dicompare antara data asli dengan data_null, hampir semua jenis data muncul (contoh: pada kolom `children` & kolom `family_status` terlihat sama pada data asli & data_null)

2. Kolom `days_employed` mengandung banyak value rancu, dimana ada yang bernilai negatif dimana seharusnya lama bekerja tidak mungkin negatif, kemudian jumlah hari kerjanya ada yang tidak masuk akal 340266 hari / 365 hari = 932 tahun. Sehingga kita perlu melakukan modifikasi yang lebih masuk akal dengan mengganti nilai negatif menjadi positif dan jumlah hari bekerja yang tidak masuk akal disesuaikan dengan umur nasabah.

3. Kolom `education` memiliki unique value yang bedanya pada gaya penulisan huruf kapital, huruf kecil, dstnya.

4. Kolom `dob_years` yang berumur 0 diperbaiki dengan mengambil umur termuda yang ada pada data, yaitu umur 19 tahun.

5. Kolom `total_income` yang hilang diperbaiki dengan mengambil nilai median, begitu pula dengan kolom `days_employed`. Nilai median dirasa lebih tepat karena value pada kolom tersebut sangatlah bervariasi.

6. Kolom `education` & kolom `purpose` merupakan data yang diinput manual sehingga memiliki banyak duplikat value sehingga diperbaiki berdasarkan jenisnya.

7. Secara umum, data memiliki banyak masalah, mulai dari data yang hilang sebesar 10%, kolom yang memiliki banyak kerancuan dan tidak masuk akal, sehingga manipulasi pada data cukup banyak untuk memperbaiki data tersebut.

8. Berdasarkan hasil analisis data:
- Pertama, memiliki anak tidak berpengaruh signifikan terhadap kegagalan pembayaran hutang, hanya selisih sedikit (8.18% -> memiliki anak vs. 7.54% -> tidak memiliki anak).
- Selanjutnya, orang-orang yang sudah menikah / pernah menikah lebih rendah memiliki % probabilitas gagal bayar dibandingkan yang belum menikah. 
- Bahkan, orang kaya dengan penghasilan besar tidak menjamin berhasil bayar tepat waktu dimana data menunjukkan golongan mereka memiliki probabilitas 50% gagal bayar hutang meski data hanya berisi 2 orang.
- Terakhir, untuk tujuan pengambilan hutang berpengaruh sampai dengan 9.36% yaitu untuk yang tujuannya membeli mobil & pendidikan.